do pliku AndroidManifest.xml
```xml
<?xml version="1.0" encoding="utf-8"?>
<manifest xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools">

    <uses-permission android:name="android.permission.INTERNET" />

    <application
    ...
    </application>

</manifest>
```

plik `build.gradle`

```kotlin
dependencies {

    // Retrofit do komunikacji sieciowej
    implementation("com.squareup.retrofit2:retrofit:3.0.0")
    // Konwerter GSON do parsowania JSON
    implementation("com.squareup.retrofit2:converter-gson:3.0.0")
    // Konwerter String do parsowania JSON
    implementation("com.squareup.retrofit2:converter-scalars:3.0.0")

    // ViewModel i Lifecycle w Compose
    implementation("androidx.lifecycle:lifecycle-viewmodel-compose:2.8.3")
    implementation("androidx.lifecycle:lifecycle-runtime-compose:2.9.4")

    ...
    }
```

```kotlin
class MainActivity : ComponentActivity() {
    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        enableEdgeToEdge()
        setContent {
            HeaderBasicExampleTheme {
                Surface(modifier = Modifier.fillMaxSize()) {
                    val repository = JokeRepository(RetrofitInstance.api)
                    val factory = JokeViewModelFactory(repository)
                    JokeScreen(viewModel = viewModel(factory = factory))
                }
            }
        }
    }
}

// =================================================================================
// --- Warstwa Danych ---
// =================================================================================

data class Joke(val id: String, val joke: String, val status: Int)
interface JokeApiService {
    @Headers("Accept: application/json")
    @GET("/")
    suspend fun getJokeAsJson(): Joke

    @Headers("Accept: text/plain")
    @GET("/")
    suspend fun getJokeAsText(): String
}

object RetrofitInstance {
    private const val BASE_URL = "https://icanhazdadjoke.com/"
    private val retrofit by lazy {
        Retrofit.Builder()
            .baseUrl(BASE_URL)
            // Ważne: dodajemy oba konwertery. Retrofit sam wybierze właściwy.
            .addConverterFactory(ScalarsConverterFactory.create()) // Do obsługi String
            .addConverterFactory(GsonConverterFactory.create()) // Do obsługi JSON
            .build()
    }
    val api: JokeApiService by lazy {
        retrofit.create(JokeApiService::class.java)
    }
}

class JokeRepository(private val apiService: JokeApiService) {
    suspend fun getJokeAsJson(): Joke = apiService.getJokeAsJson()
    suspend fun getJokeAsText(): String = apiService.getJokeAsText()
}

// =================================================================================
// --- Architektura ---
// =================================================================================

data class JokeUiState(
    val jokeText: String = "Kliknij przycisk, aby załadować żart...",
    val isLoading: Boolean = false
)

class JokeViewModel(private val repository: JokeRepository) : ViewModel() {

    private val _uiState = MutableStateFlow(JokeUiState())
    val uiState: StateFlow<JokeUiState> = _uiState.asStateFlow()

    fun fetchJsonJoke() {
        viewModelScope.launch {
            _uiState.update { it.copy(isLoading = true) }
            try {
                val jokeObject = repository.getJokeAsJson()
                _uiState.update { JokeUiState(jokeText = jokeObject.joke) }
            } catch (e: Exception) {
                _uiState.update { JokeUiState(jokeText = "Błąd: ${e.message}") }
            }
        }
    }

    fun fetchTextJoke() {
        viewModelScope.launch {
            _uiState.update { it.copy(isLoading = true) }
            try {
                val jokeText = repository.getJokeAsText()
                _uiState.update { JokeUiState(jokeText = jokeText) }
            } catch (e: Exception) {
                _uiState.update { JokeUiState(jokeText = "Błąd: ${e.message}") }
            }
        }
    }
}

class JokeViewModelFactory(private val repository: JokeRepository) : ViewModelProvider.Factory {
    override fun <T : ViewModel> create(modelClass: Class<T>): T {
        if (modelClass.isAssignableFrom(JokeViewModel::class.java)) {
            @Suppress("UNCHECKED_CAST")
            return JokeViewModel(repository) as T
        }
        throw IllegalArgumentException("Unknown ViewModel class")
    }
}


// =================================================================================
// --- UI ---
// =================================================================================

@OptIn(ExperimentalMaterial3Api::class)
@Composable
fun JokeScreen(viewModel: JokeViewModel) {
    val uiState by viewModel.uiState.collectAsStateWithLifecycle()

    Scaffold(topBar = { TopAppBar(title = { Text("Demonstracja @Headers") }) }) { padding ->
        Column(
            modifier = Modifier
                .fillMaxSize()
                .padding(padding)
                .padding(16.dp),
            horizontalAlignment = Alignment.CenterHorizontally,
            verticalArrangement = Arrangement.Center
        ) {
            Card(modifier = Modifier.defaultMinSize(minHeight = 150.dp)) {
                Box(
                    modifier = Modifier.padding(16.dp).fillMaxWidth(),
                    contentAlignment = Alignment.Center
                ) {
                    Text(
                        text = uiState.jokeText,
                        style = MaterialTheme.typography.titleMedium,
                        textAlign = TextAlign.Center
                    )
                }

            }
            Spacer(Modifier.height(24.dp))

            if (uiState.isLoading) {
                CircularProgressIndicator()
            }

            Spacer(Modifier.height(24.dp))

            Row(
                modifier = Modifier.fillMaxWidth(),
                horizontalArrangement = Arrangement.SpaceEvenly
            ) {
                Button(
                    onClick = { viewModel.fetchJsonJoke() },
                    enabled = !uiState.isLoading
                ) {
                    Text("Pobierz jako JSON")
                }
                Button(
                    onClick = { viewModel.fetchTextJoke() },
                    enabled = !uiState.isLoading
                ) {
                    Text("Pobierz jako Tekst")
                }
            }
        }
    }
}
```